# Flight Price Prediction – Complete EDA & Feature Engineering

**Objective**: Predict **flight ticket prices** using **11 input features** including airline, route, timing, stops, and class.

**Dataset**: [Kaggle – Flight Price Prediction](https://www.kaggle.com/datasets/shubhambathwal/flight-price-prediction)

## Dataset Features Explanation

The cleaned flight dataset contains several features describing airline, route, and ticket-related information. Each feature and its role are explained below:

### 1. **Airline**
- **Description:** Name of the airline operating the flight.  
- **Type:** Categorical  
- **Details:** Contains 6 unique airlines. This feature helps analyze price differences and performance between different airline companies.

### 2. **Flight**
- **Description:** Unique flight code or number associated with each journey.  
- **Type:** Categorical  
- **Details:** Represents the specific aircraft or flight identifier. Useful for tracking flight-level information but may not contribute much to predictive modeling due to high uniqueness.

### 3. **Source City**
- **Description:** The city from which the flight takes off.  
- **Type:** Categorical  
- **Details:** Has 6 distinct cities. Helps analyze route-based fare variations and demand trends from specific cities.

### 4. **Departure Time**
- **Description:** Time period when the flight departs, grouped into categorical time bins.  
- **Type:** Categorical (derived feature)  
- **Details:** Contains 6 unique time slots (e.g., Early Morning, Morning, Afternoon, Evening, Night, Late Night). Important for analyzing how departure time influences ticket prices.

### 5. **Stops**
- **Description:** Number of stops (layovers) between source and destination.  
- **Type:** Categorical  
- **Details:** Has 3 distinct categories — typically *non-stop*, *1 stop*, and *2+ stops*. Strongly affects ticket prices and total travel time.

### 6. **Arrival Time**
- **Description:** Time period when the flight arrives, grouped into categorical bins.  
- **Type:** Categorical (derived feature)  
- **Details:** Similar to *Departure Time*, it contains 6 unique time labels. Helps study how arrival time patterns vary with route and airline.

### 7. **Destination City**
- **Description:** City where the flight lands.  
- **Type:** Categorical  
- **Details:** Contains 6 unique cities. Important for studying route-based demand and pricing variations.

### 8. **Class**
- **Description:** Indicates the seat class booked for the flight.  
- **Type:** Categorical  
- **Details:** Has two categories — *Business* and *Economy*. Business class tickets are expected to be priced higher, making this a strong predictor of *Price*.

### 9. **Duration**
- **Description:** Total travel time between source and destination cities (in hours).  
- **Type:** Continuous  
- **Details:** Strongly correlated with the number of stops and distance traveled. Longer durations generally correspond to higher prices.

### 10. **Days Left**
- **Description:** Derived feature representing the number of days between booking and the travel date.  
- **Type:** Numerical (derived feature)  
- **Details:** Captures booking behavior. Fewer days left before departure typically result in higher ticket prices.

### 11. **Price**
- **Description:** The **target variable** representing the flight ticket price.  
- **Type:** Numerical  
- **Details:** This is the dependent variable used in modeling to predict flight prices based on the other features.


**Target**: `Price` (in INR)

---

## 1. Importing Required Libraries

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.preprocessing import OneHotEncoder
from scipy import stats

warnings.filterwarnings('ignore')
%matplotlib inline
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
pd.set_option('display.max_columns', None)

---

## 2. Loading the Dataset

In [4]:
df = pd.read_excel('./data/flight_price.xlsx')
print(f"Dataset: {df.shape[0]:,} rows × {df.shape[1]} columns")
df.head()

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [ ]:
df.info()

In [ ]:
df.describe()

---

## 3. Data Understanding & Cleaning

### 3.1 Missing Values

In [ ]:
missing = df.isnull().sum()
print("Missing Values:\n", missing[missing > 0])

plt.figure(figsize=(10, 4))
sns.heatmap(df.isnull(), cbar=True, yticklabels=False, cmap='magma')
plt.title('Missing Data Heatmap')
plt.show()

> **Only 1 missing in `Total_Stops`** → impute with **mode** (most frequent)

### 3.2 Duplicates

In [ ]:
dup = df.duplicated().sum()
print(f"Duplicate rows: {dup} ({dup/len(df)*100:.2f}%)")

df = df.drop_duplicates().reset_index(drop=True)
print(f"After removal: {df.shape[0]:,} rows")

---

## 4. Feature Engineering – Theory + Code

### 4.1 `Date_of_Journey` → Day, Month, Year

**Why?** Price varies by **season**, **holidays**, **weekends**

| Feature | Insight |
|--------|--------|
| `Journey_Day` | Weekend vs Weekday pricing |
| `Journey_Month` | Peak (Dec–Jan) vs Off-season |
| `Journey_Year` | All 2022 → drop later |

In [ ]:
df['Journey_Day'] = pd.to_datetime(df['Date_of_Journey'], format='%d/%m/%Y').dt.day
df['Journey_Month'] = pd.to_datetime(df['Date_of_Journey'], format='%d/%m/%Y').dt.month
df['Journey_Year'] = pd.to_datetime(df['Date_of_Journey'], format='%d/%m/%Y').dt.year

df.drop('Date_of_Journey', axis=1, inplace=True)
df[['Journey_Day', 'Journey_Month']].head()

### 4.2 `Dep_Time` & `Arrival_Time` → Hour, Minute

**Why?** Red-eye flights cheaper, peak hours expensive

In [ ]:
df['Dep_Hour'] = pd.to_datetime(df['Dep_Time']).dt.hour
df['Dep_Min'] = pd.to_datetime(df['Dep_Time']).dt.minute
df.drop('Dep_Time', axis=1, inplace=True)

df['Arrival_Hour'] = pd.to_datetime(df['Arrival_Time']).dt.hour
df['Arrival_Min'] = pd.to_datetime(df['Arrival_Time']).dt.minute
df.drop('Arrival_Time', axis=1, inplace=True)

df[['Dep_Hour', 'Arrival_Hour']].head()

### 4.3 `Total_Stops` → Numerical

**Why?** More stops = longer layover = higher operational cost → higher price

In [ ]:
stop_map = {'non-stop': 0, '1 stop': 1, '2 stops': 2, '3 stops': 3, '4 stops': 4}
df['Total_Stops'] = df['Total_Stops'].map(stop_map)
df['Total_Stops'].fillna(df['Total_Stops'].mode()[0], inplace=True)
df['Total_Stops'] = df['Total_Stops'].astype(int)

df['Total_Stops'].value_counts()

### 4.4 `Duration` → Total Minutes

**Why?** Direct proxy for fuel, crew, and aircraft cost

In [ ]:
def duration_to_min(d):
    if 'h' not in d: d = '0h ' + d
    if 'm' not in d: d = d + ' 0m'
    h = int(d.split('h')[0].strip())
    m = int(d.split('h')[-1].split('m')[0].strip())
    return h * 60 + m

df['Duration_Minutes'] = df['Duration'].apply(duration_to_min)
df.drop('Duration', axis=1, inplace=True)

df['Duration_Minutes'].head()

### 4.5 Drop Redundant

- `Route`: Redundant with `Source`, `Destination`, `Total_Stops`
- `Additional_Info`: 80% "No Info" → low signal

In [ ]:
df.drop(['Route', 'Additional_Info'], axis=1, inplace=True)
df.head()

---

## 5. Exploratory Data Analysis (EDA)

### 5.1 Univariate: Price Distribution

In [ ]:
plt.figure(figsize=(10, 5))
sns.histplot(df['Price'], bins=50, kde=True, color='teal')
plt.title('Flight Price Distribution')
plt.xlabel('Price (INR)')
plt.show()

print(f"Skewness: {df['Price'].skew():.2f}")
print(f"Kurtosis: {df['Price'].kurtosis():.2f}")

> **Highly right-skewed** → **log transform required**

### 5.2 Bivariate: Key Drivers of Price

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# Airline
sns.boxplot(x='Airline', y='Price', data=df, ax=axes[0,0], palette='Set3')
axes[0,0].set_title('Price by Airline')
axes[0,0].tick_params(axis='x', rotation=45)

# Stops
sns.boxplot(x='Total_Stops', y='Price', data=df, ax=axes[0,1])
axes[0,1].set_title('Price by Number of Stops')

# Class
sns.boxplot(x='Class', y='Price', data=df, ax=axes[1,0])
axes[1,0].set_title('Price by Class')

# Duration
sns.scatterplot(x='Duration_Minutes', y='Price', data=df, ax=axes[1,1], alpha=0.5)
axes[1,1].set_title('Price vs Flight Duration')

plt.tight_layout()
plt.show()

> **Jet Airways Business** = 5× Economy  
> **4 stops** = premium routes  
> **Business class** dominates high-end

### 5.3 Multivariate: Correlation Heatmap

In [ ]:
plt.figure(figsize=(12, 8))
corr = df.corr(numeric_only=True)
sns.heatmap(corr, annot=True, cmap='RdYlGn', center=0, fmt='.2f', square=True)
plt.title('Feature Correlation Matrix')
plt.show()

> **Top predictors**: `Duration_Minutes`, `Total_Stops`, `Class`

---

## 6. Outlier Analysis

In [ ]:
sns.boxplot(y=df['Price'], color='salmon')
plt.title('Price Outliers')
plt.show()

Q1, Q3 = df['Price'].quantile([0.25, 0.75])
IQR = Q3 - Q1
outliers = df[(df['Price'] < Q1 - 1.5*IQR) | (df['Price'] > Q3 + 1.5*IQR)]
print(f"Outliers: {len(outliers)} ({len(outliers)/len(df)*100:.1f}%)")

> **Outliers are valid** – represent **business class, last-minute bookings**

---

## 7. Normality & Transformation

In [ ]:
df['Price_Log'] = np.log1p(df['Price'])

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
sns.histplot(df['Price'], kde=True, ax=ax1, color='red')
ax1.set_title('Original Price')
sns.histplot(df['Price_Log'], kde=True, ax=ax2, color='green')
ax2.set_title('Log Transformed Price')
plt.show()

print(f"Original Skew: {df['Price'].skew():.2f} → Log Skew: {df['Price_Log'].skew():.2f}")

---

## 8. Correlation & Multicollinearity (VIF)

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

num_cols = df.select_dtypes(include=np.number).drop(['Price', 'Price_Log'], axis=1, errors='ignore')
vif = pd.DataFrame()
vif['Feature'] = num_cols.columns
vif['VIF'] = [variance_inflation_factor(num_cols.values, i) for i in range(num_cols.shape[1])]
vif = vif.sort_values('VIF', ascending=False)
print(vif)

> **High VIF**: `Duration_Minutes` and `Total_Stops` → expected (longer flight = more stops)

---

## 9. Encoding Categorical Variables

In [ ]:
cat_cols = ['Airline', 'Source', 'Destination']
encoder = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')
encoded = pd.DataFrame(encoder.fit_transform(df[cat_cols]),
                       columns=encoder.get_feature_names_out(cat_cols),
                       index=df.index)

df = pd.concat([df.drop(cat_cols, axis=1), encoded], axis=1)
print(f"Final shape: {df.shape}")

---

## 10. Final Data Preparation

In [ ]:
X = df.drop(['Price', 'Price_Log'], axis=1)
y = df['Price_Log']

print(f"X: {X.shape[1]} features, y: log(price)")
print(f"Missing in X: {X.isnull().sum().sum()}")

---

## 11. Summary of Insights & Action Plan

| Feature | Theory | Insight | Action |
|--------|-------|--------|-------|
| `Class` | Business = premium service | 5× price | **Keep + One-Hot** |
| `Airline` | Brand pricing power | Jet Airways Business = top | **One-Hot** |
| `Total_Stops` | Layover cost | More stops = higher | **Ordinal** |
| `Duration` | Fuel + crew | Strong correlation | **Minutes** |
| `Dep_Hour` | Red-eye discount | Early morning cheaper | **Keep** |
| `Journey_Month` | Seasonality | Dec–Jan peak | **Keep** |
| `Price` | Right-skewed | Log transform | **Use `Price_Log`** |

### Next Steps
1. Train **XGBoost, Random Forest**
2. Target: **Log Price** → predict `exp(y)`
3. Evaluate: **RMSE, MAE, R²**
4. Deploy with **Streamlit**

---
**End of Notebook**